In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class IdentityBlock(nn.Module):
    """
    Implementation of the identity block
    """
    def __init__(self, in_channels, filters, f, stage, block):
        super(IdentityBlock, self).__init__()
        F1, F2, F3 = filters
        
        conv_name_base = f'res{stage}{block}_branch'
        bn_name_base = f'bn{stage}{block}_branch'
        
        # First component
        self.conv1 = nn.Conv2d(in_channels, F1, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(F1)
        
        # Second component
        self.conv2 = nn.Conv2d(F1, F2, kernel_size=f, stride=1, padding=f//2, bias=False)
        self.bn2 = nn.BatchNorm2d(F2)
        
        # Third component
        self.conv3 = nn.Conv2d(F2, F3, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(F3)
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        out += identity
        out = self.relu(out)
        
        return out


class ConvolutionalBlock(nn.Module):
    """
    Implementation of the convolutional block
    """
    def __init__(self, in_channels, filters, f, stage, block, s=2):
        super(ConvolutionalBlock, self).__init__()
        F1, F2, F3 = filters
        
        conv_name_base = f'res{stage}{block}_branch'
        bn_name_base = f'bn{stage}{block}_branch'
        
        # First component of main path
        self.conv1 = nn.Conv2d(in_channels, F1, kernel_size=1, stride=s, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(F1)
        
        # Second component of main path
        self.conv2 = nn.Conv2d(F1, F2, kernel_size=f, stride=1, padding=f//2, bias=False)
        self.bn2 = nn.BatchNorm2d(F2)
        
        # Third component of main path
        self.conv3 = nn.Conv2d(F2, F3, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(F3)
        
        # Shortcut path
        self.shortcut_conv = nn.Conv2d(in_channels, F3, kernel_size=1, stride=s, padding=0, bias=False)
        self.shortcut_bn = nn.BatchNorm2d(F3)
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        identity = self.shortcut_conv(identity)
        identity = self.shortcut_bn(identity)
        
        out += identity
        out = self.relu(out)
        
        return out


class ResNet50(nn.Module):
    """
    Implementation of the ResNet50 architecture
    """
    def __init__(self, input_shape=(3, 64, 64), classes=6):
        super(ResNet50, self).__init__()
        
        # Initial layers
        self.padding = nn.ZeroPad2d(3)
        self.conv1 = nn.Conv2d(input_shape[0], 64, kernel_size=7, stride=2, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Stage 2
        self.stage2_conv = ConvolutionalBlock(64, [64, 64, 256], 3, stage=2, block='a', s=1)
        self.stage2_id1 = IdentityBlock(256, [64, 64, 256], 3, stage=2, block='b')
        self.stage2_id2 = IdentityBlock(256, [64, 64, 256], 3, stage=2, block='c')
        
        # Stage 3
        self.stage3_conv = ConvolutionalBlock(256, [128, 128, 512], 3, stage=3, block='a', s=2)
        self.stage3_id1 = IdentityBlock(512, [128, 128, 512], 3, stage=3, block='b')
        self.stage3_id2 = IdentityBlock(512, [128, 128, 512], 3, stage=3, block='c')
        self.stage3_id3 = IdentityBlock(512, [128, 128, 512], 3, stage=3, block='d')
        
        # Stage 4
        self.stage4_conv = ConvolutionalBlock(512, [256, 256, 1024], 3, stage=4, block='a', s=2)
        self.stage4_id1 = IdentityBlock(1024, [256, 256, 1024], 3, stage=4, block='b')
        self.stage4_id2 = IdentityBlock(1024, [256, 256, 1024], 3, stage=4, block='c')
        self.stage4_id3 = IdentityBlock(1024, [256, 256, 1024], 3, stage=4, block='d')
        self.stage4_id4 = IdentityBlock(1024, [256, 256, 1024], 3, stage=4, block='e')
        self.stage4_id5 = IdentityBlock(1024, [256, 256, 1024], 3, stage=4, block='f')
        
        # Stage 5
        self.stage5_conv = ConvolutionalBlock(1024, [512, 512, 2048], 3, stage=5, block='a', s=2)
        self.stage5_id1 = IdentityBlock(2048, [512, 512, 2048], 3, stage=5, block='b')
        self.stage5_id2 = IdentityBlock(2048, [512, 512, 2048], 3, stage=5, block='c')
        
        # AVGPOOL
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Output layer
        self.fc = nn.Linear(2048, classes)
        
        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.padding(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Stage 2
        x = self.stage2_conv(x)
        x = self.stage2_id1(x)
        x = self.stage2_id2(x)
        
        # Stage 3
        x = self.stage3_conv(x)
        x = self.stage3_id1(x)
        x = self.stage3_id2(x)
        x = self.stage3_id3(x)
        
        # Stage 4
        x = self.stage4_conv(x)
        x = self.stage4_id1(x)
        x = self.stage4_id2(x)
        x = self.stage4_id3(x)
        x = self.stage4_id4(x)
        x = self.stage4_id5(x)
        
        # Stage 5
        x = self.stage5_conv(x)
        x = self.stage5_id1(x)
        x = self.stage5_id2(x)
        
        # AVGPOOL
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        # Output layer
        x = self.fc(x)
        
        return x

In [7]:
from class_loading import create_data_loaders
train_loader, val_loader = create_data_loaders(2, 50)

Loading data from: C:\Datasets\ICIP training data\ICIP training data\0\RawDataQA (1)
Found 52 files


In [8]:
next(iter(train_loader))[0].shape  # To check if the data loader works

torch.Size([8, 1, 256, 256])

In [9]:
model = ResNet50(input_shape=(1, 256, 256), classes=3)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

print(model)

ResNet50(
  (padding): ZeroPad2d((3, 3, 3, 3))
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2_conv): ConvolutionalBlock(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut_conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (shortcut_bn): BatchNorm2d(256, ep

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import copy
from tqdm import tqdm

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, device=None):
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    print(f"Using device: {device}")
    model = model.to(device)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass - track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # Deep copy the model if best accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
dataloaders = {
        'train': train_loader,
        'val': val_loader
    }

num_epochs = 2

trained_model = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs)
    
# Save the trained model
torch.save(trained_model.state_dict(), 'resnet50_oct_model.pth')
print("Model saved successfully!")

Using device: cuda:0
Epoch 1/2
----------


 20%|██        | 1/5 [00:09<00:37,  9.39s/it]